In [1]:
# Se importan librerías a utillizar
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
import implicit # The Cython library
import os
import pickle
import shelve
from sklearn import metrics

# conda install -c conda-forge implicit

In [33]:
df_orders_9 = pd.read_csv("../../DataSets Grandes/PedidosYa/PedidosYa_Customer-Orders-9-2020.csv",sep=";")
df_orders_10 = pd.read_csv("../../DataSets Grandes/PedidosYa/PedidosYa_Customer-Orders-10-2020.csv",sep=";")
df_orders_11 = pd.read_csv("../../DataSets Grandes/PedidosYa/PedidosYa_Customer-Orders-11-2020.csv",sep=";")
df_customers = pd.read_csv("../../DataSets Grandes/PedidosYa/Sales_Orders order 2021-01-04T2033.csv",sep=",")

In [34]:
df_orders = pd.concat([df_orders_9,df_orders_10,df_orders_11])

In [4]:
df_categories = pd.read_excel("../../DataSets Grandes/PedidosYa/Maestro_Productos_Categorías.xlsx",header=0)

# EDA

In [5]:
df_categories.head(3)

,Product_Name,Familia
0,Duraznos En Almibar En Mitades La Banda 850 Grs.,Conservas
1,Galletitas Frutigran Salvado 500 G.,Galletas
2,Galletitas Granix Granovita Limon 140 G,Galletas


In [9]:
print("-Analizamos la cantidad de órdenes que tenemos por mes-")
print(len(df_orders_9.Order_ID.unique()),"órdenes de septiembre")
print(len(df_orders_10.Order_ID.unique()),"órdenes de octubre")
print(len(df_orders_11.Order_ID.unique()),"órdenes de noviembre")

-Analizamos la cantidad de órdenes que tenemos por mes-
50269 órdenes de septiembre
65892 órdenes de octubre
74956 órdenes de noviembre


In [35]:
#Mejoramos el nombrado de las columnas
df_orders.columns = df_orders.columns.str.replace('"', '')
df_orders.rename({"\ufeffSKU_ID":"SKU_ID"},axis=1, inplace = True)

In [11]:
print ("Cantidad de productos distintos en las órdenes:",len(df_orders.SKU_ID.unique()))

Cantidad de productos distintos en las órdenes: 2508


In [33]:
df_orders.shape

(1114173, 23)

In [12]:
df_orders.head(3)

,SKU_ID,Order_ID,Order_Date,Product_Name,Barcode_Array,Supplier_Names,Supplier_IDs,Listed_Unit_Price_New,Paid_Unit_Price_New,Unit_Discount,...,Ordered_Quantity,Fulfilled_Quantity,Returned_Quantity,Original_Item_Subtotal,Original_Item_COGS,Adjusted_Item_Subtotal,Adjusted_Item_COGS,Store_ID,Store_Name,Unit_Discount_new
0,16903473,227421127,1/9/2020 00:03,Duraznos En Almibar En Mitades La Banda 850 Grs.,"[""7790499000485"", ""07790499000485""]",Alimentos La Banda,4220,"101,82","101,8181818",0.0,...,1,1,0.0,"101,82","55,13","101,82","55,13",164044,AR_9_Microcentro,NaN
1,17315132,227421127,1/9/2020 00:03,Galletitas Frutigran Salvado 500 G.,"[""07790045001201"", ""7790045001201""]",ALIMENTOS GRANIX,7318,"101,82","101,8181818",0.0,...,1,1,0.0,"101,82","61,93","101,82","61,93",164044,AR_9_Microcentro,NaN
2,17315107,227421127,1/9/2020 00:03,Galletitas Granix Granovita Limon 140 G,"[""7790045000846"", ""07790045000846""]",ALIMENTOS GRANIX,7318,"50,18","50,18181818",0.0,...,1,1,0.0,"50,18","27,02","50,18","27,02",164044,AR_9_Microcentro,NaN


In [36]:
# Nos quedamos únicamente con las columnas siguientes:
df_customers = df_customers[["Order ID","User ID","Confirmed Orders"]]

In [37]:
# Verificamos Nulos
df_orders.isnull().sum()

SKU_ID                         0
Order_ID                       0
Order_Date                     0
Product_Name                   0
Barcode_Array                  0
Supplier_Names               871
Supplier_IDs                 871
Listed_Unit_Price_New          0
Paid_Unit_Price_New            0
Unit_Discount             383657
Unit_Cost_New                  0
Total_Amount                   0
Total_COGS                     0
Ordered_Quantity               0
Fulfilled_Quantity             0
Returned_Quantity              0
Original_Item_Subtotal         0
Original_Item_COGS             0
Adjusted_Item_Subtotal         0
Adjusted_Item_COGS             0
Store_ID                       0
Store_Name                     0
Unit_Discount_new         742765
dtype: int64

In [38]:
#Nos quedamos con las columnas que nos interesan:
df_orders_simple = df_orders.loc[:,['SKU_ID', 'Order_ID', 'Ordered_Quantity', 'Product_Name']]

In [39]:
df_orders_simple.reset_index(inplace=True, drop=True)

In [40]:
df_orders_simple.columns

Index(['SKU_ID', 'Order_ID', 'Ordered_Quantity', 'Product_Name'], dtype='object')

In [41]:
df_orders_simple.head(2)

,SKU_ID,Order_ID,Ordered_Quantity,Product_Name
0,16903473,227421127,1,Duraznos En Almibar En Mitades La Banda 850 Grs.
1,17315132,227421127,1,Galletitas Frutigran Salvado 500 G.


In [64]:
# Armamos el df con los datos de las compras y los clientes
df_full = df_orders_simple.merge(df_customers,how="inner",left_on="Order_ID",right_on="Order ID")

In [53]:
df_full.head(3)

,SKU_ID,Order_ID,Ordered_Quantity,Product_Name,Order ID,User ID,Confirmed Orders
0,16903473,227421127,1,Duraznos En Almibar En Mitades La Banda 850 Grs.,227421127,17367247,1
1,17315132,227421127,1,Galletitas Frutigran Salvado 500 G.,227421127,17367247,1
2,17315107,227421127,1,Galletitas Granix Granovita Limon 140 G,227421127,17367247,1


In [65]:
df_full = df_full.drop(columns="Order ID")

In [66]:
df_full = df_full[df_full["Confirmed Orders"] == 1]

In [67]:
df_full = df_full.drop(columns="Confirmed Orders")

In [68]:
df_full.shape

(1062964, 5)

In [69]:
df_full_cat = df_full.merge(df_categories, on="Product_Name" ,how="left")

In [70]:
df_full.SKU_ID.nunique()

2440

In [71]:
df_full.isna().sum()

SKU_ID              0
Order_ID            0
Ordered_Quantity    0
Product_Name        0
User ID             0
dtype: int64

In [72]:
df_full_cat.isna().sum()

SKU_ID                   0
Order_ID                 0
Ordered_Quantity         0
Product_Name             0
User ID                  0
Familia             281410
dtype: int64

In [79]:
print(df_full_cat["Product_Name"][df_full_cat.Familia.isna()].nunique(), "productos sin Familia")

763 productos sin Familia


In [81]:
df_full_cat.loc[:,["Product_Name","Familia"]].drop_duplicates(inplace=True)

In [83]:
df_full_cat.sample(2)

,SKU_ID,Order_ID,Ordered_Quantity,Product_Name,User ID,Familia
171768,16730456,234064174,2,Agua Mineral Sin Gas Glaciar Bajo Sodio 1.5 L.,20559978,Aguas
464014,16979858,244337563,2,Pan Arabe Con Cereales Oroweat 4 U.,6703453,Panificados


In [86]:
print(len(df_full.Order_ID), "cantidad de órdenes totales")

1062964 cantidad de órdenes totales


In [87]:
# Buscamos un valor aproximado para cortar el dataset en train y test
df_full.Order_ID.describe().apply(lambda x: format(x, 'f'))

count      1062964.000000
mean     246369855.433947
std       10540535.461057
min      227421127.000000
25%      237440125.000000
50%      246869787.000000
75%      255387802.000000
max      264691076.000000
Name: Order_ID, dtype: object

In [34]:
#df_full.loc[:,["Product_Name","Familia"]].drop_duplicates().to_csv("pepito.csv")

In [88]:
df_full_simple = df_full[['SKU_ID', 'Ordered_Quantity', 'User ID']] # Get rid of unnecessary info

In [97]:
# Agrupo los datos por lo necesario
df_full_simple_grouped = df_full_simple.groupby(["User ID","SKU_ID"]).sum().reset_index() # Group together
df_full_simple_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850661 entries, 0 to 850660
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   User ID           850661 non-null  int64
 1   SKU_ID            850661 non-null  int64
 2   Ordered_Quantity  850661 non-null  int64
dtypes: int64(3)
memory usage: 19.5 MB


In [100]:
df_full_grouped = df_full.groupby(['Order_ID', "User ID","SKU_ID"]).sum().reset_index() # Group together
df_full_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1047107 entries, 0 to 1047106
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype
---  ------            --------------    -----
 0   Order_ID          1047107 non-null  int64
 1   User ID           1047107 non-null  int64
 2   SKU_ID            1047107 non-null  int64
 3   Ordered_Quantity  1047107 non-null  int64
dtypes: int64(4)
memory usage: 32.0 MB


In [101]:
df_full_simple_grouped[df_full_simple_grouped["User ID"]==936].head(5)

,User ID,SKU_ID,Ordered_Quantity
26,936,15500909,9
27,936,15500981,1
28,936,15609034,1
29,936,15791700,5
30,936,16087199,1


In [103]:
df_full_grouped[df_full_grouped["User ID"]==936].head(3)

,Order_ID,User ID,SKU_ID,Ordered_Quantity
942035,260693088,936,15500909,4
942036,260693088,936,16405480,1
942037,260693088,936,16953366,1


In [105]:
print(len(df_full_simple['User ID'].unique()), "clientes con compras registradas")

65303 clientes con compras registradas


In [106]:
# Genero set de training y de testeo manualmente
df_full_test = df_full[df_full.Order_ID>264501076]
df_full = df_full[df_full.Order_ID<=264501076]

In [107]:
print(df_full_test.shape)
print(df_full.shape)

(2351, 5)
(1060613, 5)


In [108]:
# Create a numeric user_id and product_id column
df_full["User ID"] = df_full["User ID"].astype("category")
df_full['SKU_ID'] = df_full['SKU_ID'].astype("category")
df_full['User_ID_CODE'] = df_full['User ID'].cat.codes
df_full['SKU_ID_CODE'] = df_full['SKU_ID'].cat.codes

In [109]:
# Generamos 2 matrices esparsas, una con base cliente contra productos y la otra en base productos contra clientes
sparse_user_item = sparse.csr_matrix((df_full['Ordered_Quantity'].astype(float), (df_full['User_ID_CODE'], df_full['SKU_ID_CODE'])))
sparse_item_user = sparse.csr_matrix((df_full['Ordered_Quantity'].astype(float), (df_full['SKU_ID_CODE'], df_full['User_ID_CODE'])))

In [110]:
import implicit

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.05, iterations=10)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(sparse_item_user)

In [323]:
import implicit

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.05, iterations=10)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(product_train.T)

In [112]:
user_items = sparse_item_user.T.tocsr()

In [113]:
user_items

<65142x2440 sparse matrix of type '<class 'numpy.float64'>'
	with 848825 stored elements in Compressed Sparse Row format>

In [129]:
# Se crea función para dividir training y testing
def make_train(ratings, pct_test = 0.12):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [295]:
#Usamos la función del paper para generar el split del ds
product_train, product_test, product_users_altered = make_train(sparse_user_item, pct_test = 0.2)

In [311]:
#Con nuestro modelo entrenado en product_train, pero falla la recomendación con nuestra función por estar invertida la matrix
calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(model.user_factors), sparse.csr_matrix(model.item_factors.T)], product_test)

(0.907, 0.813)

In [325]:
#Con nuestro modelo entrenado en product_train.T
calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(model.user_factors), sparse.csr_matrix(model.item_factors.T)], product_test)

(0.718, 0.813)

In [313]:
def recommend():
    USER=  int(input("Elija el nro de cliente:"))
    CANT=  int(input("Elija la cantidad de sugerencias:"))
    # recommend items for a user
    USER_CODE = df_full.User_ID_CODE.loc[df_full["User ID"] == USER].iloc[0]
    recommendations = model.recommend(USER_CODE, user_items, N=CANT)
    print ("_______________")
    print ("Para el cliente especificado se encontraron los siguientes",CANT,"productos recomendados:")
    print ("")
    for i in np.arange(0,CANT):
        print (df_orders.Product_Name[df_orders.SKU_ID == df_full.SKU_ID.loc[df_full.SKU_ID_CODE == recommendations[i][0]].iloc[0]].iloc[0], " con un ",round(recommendations[i][1]*100,2),"%"," de probable aceptación.")

In [117]:
def similar_items():
    SKU_ID=  int(input("Elija el nro de producto:"))
    SKU_ID_IMPLICIT = df_full.SKU_ID_CODE.loc[df_full["SKU_ID"] == SKU_ID].iloc[0]
    # recommend items from other related items
    related = model.similar_items(SKU_ID_IMPLICIT)
    print ("_______________")
    print ("En base al producto elegido ("+df_orders.Product_Name[df_orders.SKU_ID==df_full.SKU_ID.loc[df_full.SKU_ID_CODE == SKU_ID_IMPLICIT].iloc[0]].values[0]+"), por compras similares se encontraron los siguientes productos:")
    print ("")
    for i in np.arange(1,10):
        print (df_orders.Product_Name[df_orders.SKU_ID == df_full.SKU_ID.loc[df_full.SKU_ID_CODE == related[i][0]].iloc[0]].iloc[0], " con un ",round(related[i][1]*100,2),"%"," de similaridad.")

In [118]:
def search_prod():
    search = input("Búsqueda de SKU por nombre:")
    print()
    return print(display(df_orders.loc[:,["SKU_ID","Product_Name"]][df_orders.Product_Name.str.contains(search)].drop_duplicates()))

In [119]:
similar_items()

Elija el nro deproducto: 16244933


_______________
En base al producto elegido (Leche Infantil Líquida NAN Etapa 2 (6 a 12 meses) Optipro 190ml), por compras similares se encontraron los siguientes productos:

Leche Fluida Infantil Sancor Bebé Etapa 2 200 Cc.  con un  93.51 %  de similaridad.
Leche fluída infantil Nidina 2 200 cc.  con un  84.97 %  de similaridad.
Leche Fluida Infantil Sancor Bebé Etapa 2 500 Cc.  con un  82.92 %  de similaridad.
Pañales Super Sec Pampers Xg 8 Un  con un  75.62 %  de similaridad.
Leche Fluida Infantil Nutrilon Etapa 2 200 Cc.  con un  72.74 %  de similaridad.
Postre Serenito Vainilla 100 gr  con un  63.12 %  de similaridad.
Pañales Confort Sec Regular Pampers Xg 8 Un  con un  61.19 %  de similaridad.
Pañales Huggies Natural Care talle G ellas 44 u.  con un  58.49 %  de similaridad.
Yogur Yogurisimo Entero Vainilla Fort Firme 120 g  con un  57.51 %  de similaridad.


In [102]:
#Esto sería para buscar el código de la categoría del producto, pero ya lo embebimos en la función de búsqueda.
#SKU_ID=  int(input("Elija el código del producto:"))
#print ("Eligió el producto: '",df_orders.Product_Name[df_orders.SKU_ID==df_full.SKU_ID.loc[df_full.SKU_ID_CODE == SKU_ID].iloc[0]].values[0],"'")
#print ("El SKU_ID del producto es el",df_full.SKU_ID.loc[df_full.SKU_ID_CODE == SKU_ID].iloc[0])

Elija el código del producto: 1221


Eligió el producto: ' Agua Mineral Villavicencio Con Gas 1,5L '
El SKU_ID del producto es el 17620665


In [120]:
#Compras por cliente
df_cltes = df_full.drop(columns=["User_ID_CODE"])
df_cltes_test = df_full_test

In [121]:
df_cltes.head(2)

,SKU_ID,Order_ID,Ordered_Quantity,Product_Name,User ID,SKU_ID_CODE
0,16903473,227421127,1,Duraznos En Almibar En Mitades La Banda 850 Grs.,17367247,910
1,17315132,227421127,1,Galletitas Frutigran Salvado 500 G.,17367247,1126


In [122]:
clte = input("Ingrese número de cliente:")
df_cltes[df_cltes["User ID"] == int(clte)].groupby(["Order_ID","Product_Name"]).sum()

Ingrese número de cliente: 17367247


Ordered_Quantity  \
Order_ID  Product_Name                                                           
227421127 Duraznos En Almibar En Mitades La Banda 850 Grs.                   1   
          Galletitas Frutigran Salvado 500 G.                                1   
          Galletitas Granix Granovita Limon 140 G                            1   
          Hebras Cuatro Quesos Light Finlandia Chef 150gr                    1   
          Huevo color Avicoper 12 un                                         1   
          Jugo En Polvo Clight Naranja Dulce 7.5 G.                          5   
          Leche Parcialmente Descremada Liviana La Sereni...                 1   
          Limón X 1Kg                                                        1   
          Manteca Milkaut 200 G.                                             1   
          Queso Fundido Adler Light 100 G.                                   1   
          Queso Por Salut Tregar Unidad Aprox. 450Grs.                       1   
          Queso crema light Tregar 300 g.                                    1   
          Tarta Quiche Brocoli y Mozzarella Huertas del P...                 1   
          Tarta Quiche Calabaza y Mozzarella Huertas del ...                 1   
          Tarta Quiche Pollo y Calabaza Huertas del Pilar...                 1   
          Tarta Quiche Ratatouille Huertas del Pilar 300g                    1   
          Tarta Quiche de Pollo y Puerro Huertas del Pila...                 1   
230772280 Acondicionador Brillo Vitaminado Fructis X 200 Cm3                 1   
          Azúcar Ledesma Clásica 1 Kg.                                       1   
          Azúcar Mascabo Arcor 500 G                                         1   
          Bastones de merluza Granja del Sol rebozados 30...                 1   
          Bolsas Para Residuos Asurín Con Manija Ajustabl...                 1   
          Galletitas Frutigran Salvado 500 G.                                1   
          Hebras Cuatro Quesos Light Finlandia Chef 150gr                    1   
          Jabón En Barra Lux Flor De Vainilla 125 G.                         1   
          Lomo De Cerdo Feteado Bocatti Horneado 70 G.                       1   
          Papas Mccain Horneables 600 gr                                     1   
          Supremitas de Pollo Granja Tres Arroyos 800 g.                     1   
          Tarta Quiche Brocoli y Mozzarella Huertas del P...                 1   
          Tarta Quiche Calabaza y Mozzarella Huertas del ...                 1   
          Tarta Quiche Pollo y Calabaza Huertas del Pilar...                 1   
          Tarta Quiche Ratatouille Huertas del Pilar 300g                    1   
          Tarta Quiche de Pollo y Puerro Huertas del Pila...                 1   

                                                              SKU_ID_CODE  
Order_ID  Product_Name                                                     
227421127 Duraznos En Almibar En Mitades La Banda 850 Grs.            910  
          Galletitas Frutigran Salvado 500 G.                        1126  
          Galletitas Granix Granovita Limon 140 G                    1110  
          Hebras Cuatro Quesos Light Finlandia Chef 150gr             756  
          Huevo color Avicoper 12 un                                 1936  
          Jugo En Polvo Clight Naranja Dulce 7.5 G.                    47  
          Leche Parcialmente Descremada Liviana La Sereni...           48  
          Limón X 1Kg                                                 653  
          Manteca Milkaut 200 G.                                      819  
          Queso Fundido Adler Light 100 G.                            783  
          Queso Por Salut Tregar Unidad Aprox. 450Grs.               1175  
          Queso crema light Tregar 300 g.                             161  
          Tarta Quiche Brocoli y Mozzarella Huertas del P...         1088  
          Tarta Quiche Calabaza y Mozzarella Huerta

# Guardo en pkl

In [123]:
with open('modelo_implicit.pkl', 'wb') as m_imp:
    pickle.dump(model, m_imp)

In [124]:
with open('df_full.pkl', 'wb') as df_principal:
    pickle.dump(df_full, df_principal)

In [125]:
with open('user_items.pkl', 'wb') as user_items_matrix:
    pickle.dump(user_items, user_items_matrix)

## Modelo de Paper

In [127]:
# Genero una listas de cada cliente, producto y cantidades y luego una matriz esparsa con todo lo anterior
customers = list(np.sort(df_full_simple_grouped["User ID"].unique())) # Get our unique customers
products = list(df_full_simple_grouped.SKU_ID.unique()) # Get our unique products that were purchased
quantity = list(df_full_simple_grouped.Ordered_Quantity) # All of our purchases
#familia = list(df_full_simple_grouped.Familia)

rows = df_full_simple_grouped["User ID"].astype('category').cat.codes     #, categories = customers).cat.codes 
# Get the associated row indices
cols = df_full_simple_grouped.SKU_ID.astype('category').cat.codes   #, categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [128]:
# Evalúo la el nivel de la sparsidad (sparsity)
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
print("sparsity:",round(sparsity,2),"%")
print ("El tope para la sparsidad en collaborative filtering es de 99,5%, quedamos con lo justo")

sparsity: 99.47 %
El tope para la sparsidad en collaborative filtering es de 99,5%, quedamos con lo justo


In [130]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [131]:
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*15).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

## Primer modelo considera órdenes, segundo modelo solo considera las compras de cada cliente en producto y cantidad independientemente de lo que se haya comprado en conjunto

## Este modelo en lugar de instanciarse como modelo, se guarda los vectores de clientes y productos comprados

In [132]:
# Se define una variable con los datos de los productos para hacer la búsqueda de los nombres
item_lookup = df_orders[['SKU_ID', 'Product_Name']].drop_duplicates() # Only get unique item/description pair

In [133]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [134]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
         
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [64]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.77, 0.812)

In [135]:
customers_arr = pd.unique(np.array(df_full_simple_grouped["User ID"])) # Array of customer IDs from the ratings matrix
products_arr = pd.unique(np.array(df_full_simple_grouped.SKU_ID)) # Array of product IDs from the ratings matrix

In [136]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.SKU_ID.isin(prod_codes)]

In [281]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Product_Name.loc[item_lookup.SKU_ID == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'StockCode': codes, 'Description': descriptions}) # Create a dataframe 
    return final_frame[['StockCode', 'Description']] # Switch order of columns around

In [274]:
def get_customer_purch_test():
    user_id = int(input("Ingrese número de ciente:"))
    res = df_cltes_test[df_cltes_test["User ID"]==user_id].groupby(by=["Product_Name"]).sum()
    return res

In [248]:
def get_customer_purch():
    user_id = int(input("Ingrese número de ciente:"))
    res = df_cltes[df_cltes["User ID"]==user_id].groupby(by=["Product_Name"]).sum()
    return res

In [139]:
customers_arr[:20]

array([ 422,  647,  936, 1310, 1414, 1499, 1623, 1669, 1889, 2203, 2407,
       2623, 2660, 3017, 3615, 4256, 4395, 4495, 4504, 5191], dtype=int64)

# Probamos las recomendaciones con el modelo inicial

## Tenemos una serie de clientes dentro del set de test

In [254]:
df_cltes_test["User ID"].unique()[:30]

array([17766784,  5472483,  1813466, 25472616, 26514435, 16091522,
        3581030,  6337141,  6527116,  4580056,  3777079,  4685492,
        5042860,  3552449,  7050854,  5990542,  6619960, 20346744,
        4315340, 18719860,  6505868,   456721,  4687696,  5664218,
       21140825, 16152298,  1299452,   841007,  2095553, 16208968],
      dtype=int64)

## Por ejemplo este cliente compró esta serie de artículos (listados 15)

In [170]:
get_customer_purch().sort_values("Ordered_Quantity", ascending=False)[:15]

Ingrese número de ciente: 18719860


,SKU_ID,Order_ID,Ordered_Quantity,Product_Name,User ID,SKU_ID_CODE
562810,16683301,247586383,3,Galletitas Pepitos 118 G.,18719860,711
562808,21995769,247586383,2,Fajitas Tia Maruca Clásicas 150 g,18719860,2190
367855,15791700,240628138,2,Gaseosa Schweppes Pomelo Sin Azúcar 1.5 L.,18719860,211
563203,15500981,247590748,1,Papel Higiénico Hoja Simple Elegante Aloe Vera...,18719860,55
563202,21432789,247590748,1,Limpiador Líquido Ecovita Baño 550 Ml,18719860,1988
562813,15609031,247586383,1,Leche Larga Vida Entera Tregar 1 L.,18719860,146
563204,16206095,247590748,1,Rollo De Cocina Elegante 3 X 50 Paños,18719860,482
562814,16904148,247586383,1,Levadura Instántanea en Sobre Mi Pan 10 g,18719860,926
562815,17076187,247586383,1,Pan Blanco Bimbo 550 G,18719860,1057
562816,18762362,247586383,1,Pan De Hamburguesa Mendia 210 Gr,18719860,1418


In [171]:
recommend() #16999, 61726

Elija el nro de cliente: 18719860
Elija la cantidad de sugerencias: 7


_______________
Para el cliente especificado se encontraron los siguientes 7 productos recomendados:

Queso Rallado Tregar 120 G.  con un  36.44 %  de probable aceptación.
Soda Cordoba Sifon 2L  con un  33.09 %  de probable aceptación.
Jamón cocido feteado La Comarca 200 g.  con un  28.4 %  de probable aceptación.
Leche Larga Vida Entera Milkaut 1 L.  con un  26.89 %  de probable aceptación.
Hamburguesas de Carne Vacuna Clasica Paty Caja 4Un 320Grs.  con un  25.58 %  de probable aceptación.
Galletitas Chocolinas 250 G.  con un  25.18 %  de probable aceptación.
Bizcochos Don Satur Salado 200 G  con un  25.17 %  de probable aceptación.


## Verificamos en set test los productos que terminó comprando el resto de noviembre

In [229]:
get_customer_purch_test().sort_values("Ordered_Quantity", ascending=False)[:15]

Ingrese número de ciente: 18719860


,SKU_ID,Order_ID,Ordered_Quantity,Product_Name,User ID
1072077,16037187,264506916,1,Capelletini Giacomo De Verdura Fortificado 500 G.,18719860
1072078,16730558,264506916,1,Cerveza Andes Origen ROJA Lata 473ml,18719860
1072079,16730552,264506916,1,Cerveza Patagonia AMBER LAGER Lata 473ml,18719860
1072080,16730549,264506916,1,Cerveza Patagonia BOHEMIAN PILSENER Botella 730ml,18719860
1072081,16730553,264506916,1,Cerveza Patagonia KÜNÉ Lata 473ml,18719860
1072082,15791700,264506916,1,Gaseosa Schweppes Pomelo Sin Azúcar 1.5 L.,18719860
1072083,15791699,264506916,1,Gaseosa Schweppes tónica 1.5 l.,18719860
1072084,20404766,264506916,1,Huevos color Avicoper 6 u.,18719860
1072085,16730452,264506916,1,Leche Larga Vida Descremada Milkaut 1 L.,18719860
1072086,16087209,264506916,1,Mozzarella La Serenisima 220gr,18719860


## Se le recomiendan los siguientes

In [150]:
recommend() #18037034

Elija el nro de cliente: 18037034
Elija la cantidad de sugerencias: 5


_______________
Para el cliente especificado se encontraron los siguientes 5 productos recomendados:

Leche Parcialmente Descremada Liviana La Serenisima Botella Larga Vida 1L  con un  14.27 %  de probable aceptación.
Leche Zero Lactosa La Serenisima Botella Larga Vida 1L  con un  11.61 %  de probable aceptación.
Leche Clasica mas Liviana La Serenisima Botella Larga Vida 1L  con un  11.39 %  de probable aceptación.
Leche Descremada Menos Calorias La Serenisima Botella Larga Vida 1L  con un  10.37 %  de probable aceptación.
Galletitas Macucas Chocolate Relleno De Vainilla 123 G.  con un  10.37 %  de probable aceptación.


In [173]:
print("Vemos que se le recomendaron productos como Leche Larga Vida Descremada Milkaut 1 L, que efectivamente ha comprado")

Vemos que se le recomendaron productos como Leche Larga Vida Descremada Milkaut 1 L, que efectivamente ha comprado


## Verificamos usuarios similares

In [252]:
def sim_users_code():
    clte = input("Ingrese el número de cliente a evaluar:")
    clte_mod = df_full.User_ID_CODE.loc[df_full["User ID"] == int(clte)].iloc[0]
    print("Los 3 clientes más similares son:")
    sim_user_list = [x[0] for x in model.similar_users(clte_mod,4)[1:]]
    return [df_full["User ID"].loc[df_full["User_ID_CODE"] == int(x)].iloc[0] for x in sim_user_list]

In [256]:
#Clientes de train set
df_cltes["User ID"].unique()[:10]

[17367247, 18819033, 16938231, 877052, 15263554, 3037126, 3149600, 15289390, 322482, 4419610]
Categories (65142, int64): [17367247, 18819033, 16938231, 877052, ..., 17782524, 6157598, 3559295, 5810675]

In [264]:
sim_users_code() #322482

Ingrese el número de cliente a evaluar: 322482


Los 3 clientes más similares son:


[1599640, 11885491, 16734514]

In [263]:
get_customer_purch().sort_values("Ordered_Quantity", ascending=False)[:8]

Ingrese número de ciente: 322482


,Order_ID,Ordered_Quantity,SKU_ID_CODE
Product_Name,,,
Bebida isotónica Gatorade manzana 750 cc.,227424882,3,876
Café Instántaneo Arlistán Suave 170 G.,227424882,1,995
Galletitas Pepas Terepín 400 G.,227424882,1,314
Papel Higiénico Hoja Simple Elegante 4 X 30 M.,227424882,1,56
Pasta dental Colgate triple acción 90 g.,227424882,1,16
Yerba Mate Unión Suave 1 Kg.,227424882,1,253


In [265]:
get_customer_purch().sort_values("Ordered_Quantity", ascending=False)[:8]

Ingrese número de ciente: 1599640


,Order_ID,Ordered_Quantity,SKU_ID_CODE
Product_Name,,,
2 X $99 - Chocolate Cadbury Con Yogur De Frutilla X 27 Grs.,243505587,1,1357
Bebida isotónica Gatorade limón 500 cc.,251043586,1,352
Bebida isotónica Gatorade manzana 750 cc.,251043586,1,876
Galletitas Crackers Mediatarde 3 X 110 G.,251043586,1,2188
Papel Higiénico Hoja Simple Elegante 4 X 30 M.,251043586,1,56
Queso Por Salut Light Tregar Unidad Aprox. 450Grs.,251043586,1,1178
Shampoo Garnier Fructis Liso Fragancia De Coco 200 Cc.,243505587,1,492
Té Taragui Tilo 10 Sobres,243505587,1,1270


In [267]:
sim_users_code()

Ingrese el número de cliente a evaluar: 4419610


Los 3 clientes más similares son:


[19569454, 24962232, 22223104]

In [268]:
get_customer_purch().sort_values("Ordered_Quantity", ascending=False)[:8]

Ingrese número de ciente: 4419610


,Order_ID,Ordered_Quantity,SKU_ID_CODE
Product_Name,,,
Jamón Cocido Natural Feteado Bocatti 130 G.,2703823633,11,13233
Jugo Citric Naranja Con Pulpa 1 L.,1286275298,9,5995
Salame Fuet Bocatti Picado Fino 130 G.,1455396913,7,7236
Cerveza rubia Sol 473 cc.,262069895,5,1083
Queso Cremoso Tregar Unidad Aprox. 450Grs.,1226575362,5,5900
Crema De Leche Milkaut 360 Cc.,698918999,5,2475
Queso Holanda Tregar Unidad Aprox. 300Grs,995975232,5,4732
Gaseosa Coca Cola 2.25 l.,726633522,4,21


In [269]:
get_customer_purch().sort_values("Ordered_Quantity", ascending=False)[:8]

Ingrese número de ciente: 19569454


,Order_ID,Ordered_Quantity,SKU_ID_CODE
Product_Name,,,
Jamón Cocido Natural Feteado Bocatti 130 G.,714932955,6,3609
Mozzarella La Serenisima 220gr,469837106,4,774
Queso Criollo Tregar Unidad Aprox. 320Grs.,714932955,3,3528
Tapas para empanadas La Salteña 12 u. sin TACC,476416249,3,2762
Queso Crematto Milkaut 290 G.,231320400,2,822
Bizcochos De Arroz Gallo Snacks Dulces 100 G.,462934667,2,568
Cerveza rubia Amstel 473 cc.,231614267,2,630
Toalla Higienica Normal Con A Seca Always X 8 Cu,231614267,2,1762


# Validamos con el modelo del paper

In [271]:
customers_test_arr = pd.unique(np.array(df_cltes_test["User ID"])) # Array of customer IDs from the ratings matrix
customers_test_arr[:30]

array([17766784,  5472483,  1813466, 25472616, 26514435, 16091522,
        3581030,  6337141,  6527116,  4580056,  3777079,  4685492,
        5042860,  3552449,  7050854,  5990542,  6619960, 20346744,
        4315340, 18719860,  6505868,   456721,  4687696,  5664218,
       21140825, 16152298,  1299452,   841007,  2095553, 16208968],
      dtype=int64)

In [272]:
get_items_purchased(16999, product_train, customers_arr, products_arr, item_lookup)[:10] #61726, 806441, 190021, 1414

,SKU_ID,Product_Name
574,17315055,Barrita de cereal Granix 5 u.
751,18977602,Dulce batata Arcor 500g
861,16730478,Queso Danbo Adler En Fetas 144 G.
1689,18977851,Galletitas Maná Livianas Vainilla 145 G.
1746,15607212,Café Instantáneo Dolca Suave 170 G.
1842,18977769,Gomitas Mogul Moras 150 G.
2010,19643503,Pañal Xxg Pants Pampers X 16 Cu
2545,16087229,Leche Protein Con Chocolate La Serenisima Pris...
6643,15527702,Café Instantáneo Dolca 170 G.
10483,15527701,Puré De Papas Instantáneo Maggi 125 G.


In [285]:
rec_items(16999, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 5)

,StockCode,Description
0,24619361,Gaseosa Coca Cola 1.5 L.
1,15527724,Cepillo Dental Colgate Triple Acción 2 U.
2,17849066,Toallitas Húmedas Antibacteriales Espadol Dett...
3,15791705,Agua saborizada Aquarius pomelo 1.5 l.
4,21432884,Crema Dental Noc 10 Concentrada 70 G.


In [284]:
get_items_purchased(16999, product_test, customers_arr, products_arr, item_lookup)[:10]

,SKU_ID,Product_Name
369,16037183,Dulce de Leche Colonial La Serenisima 400gr
439,16683321,Chocolate Shot Con Maní 35 G.
482,18977862,Galletitas Crackers Traviata Sandwich 3 X 101 G.
574,17315055,Barrita de cereal Granix 5 u.
751,18977602,Dulce batata Arcor 500g
861,16730478,Queso Danbo Adler En Fetas 144 G.
1689,18977851,Galletitas Maná Livianas Vainilla 145 G.
1746,15607212,Café Instantáneo Dolca Suave 170 G.
1842,18977769,Gomitas Mogul Moras 150 G.
2010,19643503,Pañal Xxg Pants Pampers X 16 Cu


# Este modelo no parece haber predecido tan bien..
# Habría que modificar la de ROC AUC para nuestro modelo y ver qué sale

## Fuente importante para tener en cuenta a la hora de explicar y detallar: https://jessesw.com/Rec-System/
## https://github.com/benfred/implicit